In [155]:
import numpy as np 
import time 
from useful_function_joint import *
import pymanopt
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem
from typing import Sequence
from scipy.spatial.transform import Rotation as R

import networkx as nx

In [255]:
scale_shift = 100

In [256]:
e1 = np.array((1,0,0))*scale_shift
e2 = np.array((0,1,0))*scale_shift
e3 = np.array((0,0,1))*scale_shift
e0 = np.array([0,0,0])

In [257]:
t1 = np.array((25,0,0))
t2 = np.array((560,0,0))
t3 = np.array((515/2,0,25))
t4 = np.array((515/2,0,0))
t5 = np.array((90,0,0))

In [258]:
joints_ref = np.array([0,-90,90,0,0,0])*np.pi/180

In [259]:
R1 = Rz(joints_ref[0])
R2 = Ry(joints_ref[1])
R3 = Ry(joints_ref[2])
R4 = Rx(joints_ref[3])
R5 = Ry(joints_ref[4])
R6 = Rx(joints_ref[5])

In [260]:
T1 = T(t1)
T2 = T(t2)
T3 = T(t3)
T4 = T(t4)
T5 = T(t5)

In [261]:
axes = ['z','y','y','x','y','x']
translation = [T(e0),T1,T2,T3,T4,T5]
rotation = [R1,R2,R3,R4,R5,R6]

step_vect = {'z': [e0,e3],
             'y': [e2,-e2],
             'x': [e0,-e1]}

In [262]:
position = pose_robot_(joints_ref,'camera')[:3,3]
#np.array([400,0,695])
direction = np.array([1,0,0.])#/np.sqrt(2)

G = nx.Graph()

G.add_node(1,pos_ref=e1)
G.add_node(2,pos_ref=e2)
G.add_edge(1,2, weight  = np.linalg.norm(e1-e2))


count = 2

for i, ax in enumerate(axes):
    v1,v2 = step_vect[ax]

    transform = []

    for j in range(i+1):
        transform.append(translation[j])
        transform.append(rotation[j])
    transform = np.array(transform)
    transformation = np.linalg.multi_dot(transform)

    G.add_node(count+1,pos_ref=(transformation@T(v2))[:3,3])
    G.add_node(count+2,pos_ref=(transformation@T(v1))[:3,3])

    node_pos_ref = nx.get_node_attributes(G,'pos_ref')

    G.add_edge(count+2,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count+2]))

    G.add_edge(count,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
    G.add_edge(count-1,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))

    G.add_edge(count,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
    G.add_edge(count-1,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))

    count = count + 2

mapping = {1: "x", 2: "y", 3: "a1_plus", 4: "a1_origin", 5: "a2_minus", 6: " a2_plus", 7: "a3_minus", 8:"a3_plus", 
           9: "a4_minus", 10:"a4_origin", 11: "a5_minus", 12:"a5_plus", 13: "a6_minus", 14:"a6_origin"}
G = nx.relabel_nodes(G, mapping)

#G.add_edge('a1_origin','a6_origin', weight = np.linalg.norm(position))

G.add_edge('x','a6_origin', weight = np.linalg.norm(position-e1))
G.add_edge('y','a6_origin', weight = np.linalg.norm(position-e2))
G.add_edge('a1_plus','a6_origin', weight = np.linalg.norm(position-e3))

  
G.add_node('direction',pos_ref=(transformation@T(e1/scale_shift))[:3,3])

G.add_edge('a6_origin','direction', weight = np.linalg.norm(direction))

G.add_edge('x','direction', weight = np.linalg.norm(position+direction-e1))
G.add_edge('y','direction', weight = np.linalg.norm(position+direction-e2))
G.add_edge('a1_plus','direction', weight = np.linalg.norm(position+direction-e3))
  

In [263]:
#G.add_edge('a1_origin','a6_minus', weight = np.linalg.norm(position-scale_shift*direction))

G.add_edge('x','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e1))
G.add_edge('y','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e2))
G.add_edge('a1_plus','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e3))



In [264]:
W = np.ones((41,))
list_ = ['x','y','a1_origin','position','direction']
for i, e in enumerate(list(G.edges)):
    if e[0] in list_ or e[1] in list_:
        W[i] = 10
W = np.diag(W)

In [265]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_plus'), ('x', 'a1_origin'), ('x', 'a6_origin'), ('x', 'direction'), ('x', 'a6_minus'), ('y', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a6_origin'), ('y', 'direction'), ('y', 'a6_minus'), ('a1_plus', 'a1_origin'), ('a1_plus', 'a2_minus'), ('a1_plus', ' a2_plus'), ('a1_plus', 'a6_origin'), ('a1_plus', 'direction'), ('a1_plus', 'a6_minus'), ('a1_origin', 'a2_minus'), ('a1_origin', ' a2_plus'), ('a2_minus', ' a2_plus'), ('a2_minus', 'a3_minus'), ('a2_minus', 'a3_plus'), (' a2_plus', 'a3_minus'), (' a2_plus', 'a3_plus'), ('a3_minus', 'a3_plus'), ('a3_minus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_plus', 'a4_origin'), ('a4_minus', 'a4_origin'), ('a4_minus', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_origin', 'a5_plus'), ('a5_minus', 'a5_plus'), ('a5_minus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_plus', 'a6_origin'), ('a6_minus', 'a6_origin'), ('a6_origin', 'direction')])

In [266]:
nx.get_node_attributes(G,'pos_ref')

{'x': array([100,   0,   0]),
 'y': array([  0, 100,   0]),
 'a1_plus': array([  0.,   0., 100.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([  25., -100.,    0.]),
 ' a2_plus': array([ 25., 100.,   0.]),
 'a3_minus': array([  25., -100.,  560.]),
 'a3_plus': array([ 25., 100., 560.]),
 'a4_minus': array([182.5,   0. , 585. ]),
 'a4_origin': array([282.5,   0. , 585. ]),
 'a5_minus': array([ 540., -100.,  585.]),
 'a5_plus': array([540., 100., 585.]),
 'a6_minus': array([530.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.]),
 'direction': array([631.,   0., 585.])}

In [267]:
Y_init = np.zeros((3,41))

for i, e in enumerate(list(G.edges)):
    print(e[0])
    p1 = G.nodes[e[0]]['pos_ref']
    p2 = G.nodes[e[1]]['pos_ref']
    Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

Y_init.shape

x
x
x
x
x
x
y
y
y
y
y
a1_plus
a1_plus
a1_plus
a1_plus
a1_plus
a1_plus
a1_origin
a1_origin
a2_minus
a2_minus
a2_minus
 a2_plus
 a2_plus
a3_minus
a3_minus
a3_minus
a3_plus
a3_plus
a4_minus
a4_minus
a4_minus
a4_origin
a4_origin
a5_minus
a5_minus
a5_minus
a5_plus
a5_plus
a6_minus
a6_origin


(3, 41)

In [268]:

cost(Y_init)


29604.905800385444

In [269]:
nx.get_node_attributes(G,'pos_ref')


{'x': array([100,   0,   0]),
 'y': array([  0, 100,   0]),
 'a1_plus': array([  0.,   0., 100.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([  25., -100.,    0.]),
 ' a2_plus': array([ 25., 100.,   0.]),
 'a3_minus': array([  25., -100.,  560.]),
 'a3_plus': array([ 25., 100., 560.]),
 'a4_minus': array([182.5,   0. , 585. ]),
 'a4_origin': array([282.5,   0. , 585. ]),
 'a5_minus': array([ 540., -100.,  585.]),
 'a5_plus': array([540., 100., 585.]),
 'a6_minus': array([530.,   0., 585.]),
 'a6_origin': array([630.,   0., 585.]),
 'direction': array([631.,   0., 585.])}

In [270]:
G.edges()
nx.get_edge_attributes(G, "weight")

{('x', 'y'): 141.4213562373095,
 ('x', 'a1_plus'): 141.4213562373095,
 ('x', 'a1_origin'): 100.0,
 ('x', 'a6_origin'): 789.3826701923472,
 ('x', 'direction'): 790.0544285047708,
 ('x', 'a6_minus'): 726.0337457721921,
 ('y', 'a1_plus'): 141.4213562373095,
 ('y', 'a1_origin'): 100.0,
 ('y', 'a6_origin'): 865.520074868284,
 ('y', 'direction'): 866.2482323214288,
 ('y', 'a6_minus'): 795.6915231419774,
 ('a1_plus', 'a1_origin'): 100.0,
 ('a1_plus', 'a2_minus'): 143.61406616345073,
 ('a1_plus', ' a2_plus'): 143.61406616345073,
 ('a1_plus', 'a6_origin'): 795.0628905941969,
 ('a1_plus', 'direction'): 795.8555145250926,
 ('a1_plus', 'a6_minus'): 718.4184017687743,
 ('a1_origin', 'a2_minus'): 103.07764064044152,
 ('a1_origin', ' a2_plus'): 103.07764064044152,
 ('a2_minus', ' a2_plus'): 200.0,
 ('a2_minus', 'a3_minus'): 560.0,
 ('a2_minus', 'a3_plus'): 594.6427498927402,
 (' a2_plus', 'a3_minus'): 594.6427498927402,
 (' a2_plus', 'a3_plus'): 560.0,
 ('a3_minus', 'a3_plus'): 200.0,
 ('a3_minus', '

In [272]:

pos_ref = nx.get_node_attributes(G,'pos_ref')

weight = nx.get_edge_attributes(G, "weight")
D = np.diag([weight[i] for i in weight.keys()])
C = incidence_matrix_(G,oriented=True).toarray()
C = np.array(C)
print(C.shape)
d = 3
n = D.shape[0]

manifold_ = manifolds.Oblique(d,n)

W = np.eye(n)

Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D;
Q1 = D@W@D;

@pymanopt.function.numpy(manifold_)
def cost(Y_):
    return np.trace(Q1 + Q2@Y_.transpose()@Y_) 

@pymanopt.function.numpy(manifold_)
def euclidean_gradient(Y_):
    grad = 2*Q2@Y_.transpose()
    return grad.transpose() 
                        
@pymanopt.function.numpy(manifold_)
def euclidean_hessian(Y_, U):
    hess = 2*Q2@U.transpose()
    return hess.transpose()

problem = Problem(manifold=manifold_, cost=cost,euclidean_gradient=euclidean_gradient, 
                  euclidean_hessian=euclidean_hessian)

optimizer = optimizers.TrustRegions(max_iterations=500)

Y_init_ = np.zeros((d,41))
h = manifold_.random_point()
Y_init_[:d,:] =  h #Y_star#h#manifold_.retraction(Y_init,0.01*manifold_.random_tangent_vector(Y_init))#Y_init#h#manifold_.retraction(h,10*manifold_.random_tangent_vector(h))#Y_init
start = time.time()
Y_star = optimizer.run(problem,initial_point=Y_init_).point
end = time.time()
end-start

(15, 41)
Optimizing...
                                            f: +5.819269e+06   |grad|: 9.653081e+05
acc       k:     1     num_inner:     0     f: +3.953268e+06   |grad|: 7.807144e+05   negative curvature
acc       k:     2     num_inner:     0     f: +2.695039e+06   |grad|: 6.874781e+05   exceeded trust region
acc       k:     3     num_inner:     0     f: +1.983236e+06   |grad|: 7.011931e+05   exceeded trust region
acc       k:     4     num_inner:     1     f: +1.506794e+06   |grad|: 9.865467e+05   negative curvature
acc       k:     5     num_inner:     1     f: +5.667299e+05   |grad|: 5.651875e+05   exceeded trust region
acc TR+   k:     6     num_inner:     2     f: +2.110137e+05   |grad|: 2.285239e+05   exceeded trust region
acc       k:     7     num_inner:     5     f: +1.372108e+05   |grad|: 2.206873e+05   exceeded trust region
acc       k:     8     num_inner:     9     f: +6.869147e+04   |grad|: 1.570858e+05   exceeded trust region
acc TR-   k:     9     num_inner:  

acc       k:    86     num_inner:    81     f: +2.034372e-02   |grad|: 1.978653e-01   maximum inner iterations
acc       k:    87     num_inner:    81     f: +2.032867e-02   |grad|: 3.025532e-01   maximum inner iterations
acc       k:    88     num_inner:    81     f: +2.031428e-02   |grad|: 2.344037e-01   maximum inner iterations
acc       k:    89     num_inner:    81     f: +2.030039e-02   |grad|: 3.736627e-01   maximum inner iterations
acc       k:    90     num_inner:    81     f: +2.028700e-02   |grad|: 1.276439e-01   maximum inner iterations
acc       k:    91     num_inner:    81     f: +2.027446e-02   |grad|: 1.301082e-01   maximum inner iterations
acc       k:    92     num_inner:    81     f: +2.026192e-02   |grad|: 5.641291e-01   maximum inner iterations
acc       k:    93     num_inner:    81     f: +2.024950e-02   |grad|: 2.212407e-01   maximum inner iterations
acc       k:    94     num_inner:    81     f: +2.023807e-02   |grad|: 1.084993e-01   maximum inner iterations
a

acc       k:   171     num_inner:    81     f: +1.992967e-02   |grad|: 4.836391e-02   maximum inner iterations
acc       k:   172     num_inner:    81     f: +1.992825e-02   |grad|: 7.566104e-02   maximum inner iterations
acc       k:   173     num_inner:    81     f: +1.992686e-02   |grad|: 5.757515e-02   maximum inner iterations
acc       k:   174     num_inner:    81     f: +1.992546e-02   |grad|: 1.482198e-01   maximum inner iterations
acc       k:   175     num_inner:    81     f: +1.992408e-02   |grad|: 2.175110e-02   maximum inner iterations
acc       k:   176     num_inner:    81     f: +1.992278e-02   |grad|: 7.625096e-02   maximum inner iterations
acc       k:   177     num_inner:    81     f: +1.992147e-02   |grad|: 9.080373e-02   maximum inner iterations
acc       k:   178     num_inner:    81     f: +1.992019e-02   |grad|: 2.922211e-02   maximum inner iterations
acc       k:   179     num_inner:    81     f: +1.991894e-02   |grad|: 3.442664e-02   maximum inner iterations
a

acc       k:   252     num_inner:    81     f: +1.984133e-02   |grad|: 8.545775e-02   maximum inner iterations
acc       k:   253     num_inner:    81     f: +1.984025e-02   |grad|: 1.437190e-01   maximum inner iterations
acc       k:   254     num_inner:    81     f: +1.983917e-02   |grad|: 7.883456e-02   maximum inner iterations
acc       k:   255     num_inner:    81     f: +1.983812e-02   |grad|: 4.007200e-02   maximum inner iterations
acc       k:   256     num_inner:    81     f: +1.983708e-02   |grad|: 4.697255e-02   maximum inner iterations
acc       k:   257     num_inner:    81     f: +1.983604e-02   |grad|: 1.037197e-01   maximum inner iterations
acc       k:   258     num_inner:    81     f: +1.983500e-02   |grad|: 4.478722e-02   maximum inner iterations
acc       k:   259     num_inner:    81     f: +1.983396e-02   |grad|: 4.605985e-02   maximum inner iterations
acc       k:   260     num_inner:    81     f: +1.983291e-02   |grad|: 3.695087e-02   maximum inner iterations
a

acc       k:   330     num_inner:    81     f: +1.975211e-02   |grad|: 3.127878e-02   maximum inner iterations
acc       k:   331     num_inner:    81     f: +1.975083e-02   |grad|: 5.448020e-02   maximum inner iterations
acc       k:   332     num_inner:    81     f: +1.974955e-02   |grad|: 4.967891e-02   maximum inner iterations
acc       k:   333     num_inner:    81     f: +1.974827e-02   |grad|: 7.905629e-02   maximum inner iterations
acc       k:   334     num_inner:    81     f: +1.974699e-02   |grad|: 4.089067e-02   maximum inner iterations
acc       k:   335     num_inner:    81     f: +1.974571e-02   |grad|: 5.073763e-02   maximum inner iterations
acc       k:   336     num_inner:    81     f: +1.974443e-02   |grad|: 6.708006e-02   maximum inner iterations
acc       k:   337     num_inner:    81     f: +1.974315e-02   |grad|: 9.263257e-02   maximum inner iterations
acc       k:   338     num_inner:    81     f: +1.974185e-02   |grad|: 5.081424e-02   maximum inner iterations
a

acc       k:   411     num_inner:    81     f: +1.962543e-02   |grad|: 2.710304e-02   maximum inner iterations
acc       k:   412     num_inner:    81     f: +1.962354e-02   |grad|: 1.200496e-01   maximum inner iterations
acc       k:   413     num_inner:    81     f: +1.962165e-02   |grad|: 5.894463e-02   maximum inner iterations
acc       k:   414     num_inner:    81     f: +1.961971e-02   |grad|: 1.496770e-01   maximum inner iterations
acc       k:   415     num_inner:    81     f: +1.961776e-02   |grad|: 5.397648e-02   maximum inner iterations
acc       k:   416     num_inner:    81     f: +1.961583e-02   |grad|: 8.019940e-02   maximum inner iterations
acc       k:   417     num_inner:    81     f: +1.961388e-02   |grad|: 8.552941e-02   maximum inner iterations
acc       k:   418     num_inner:    81     f: +1.961189e-02   |grad|: 1.355944e-01   maximum inner iterations
acc       k:   419     num_inner:    81     f: +1.960990e-02   |grad|: 8.545271e-02   maximum inner iterations
a

acc       k:   486     num_inner:    81     f: +1.923487e-02   |grad|: 3.037274e-01   maximum inner iterations
acc       k:   487     num_inner:    81     f: +1.922289e-02   |grad|: 8.675174e-02   maximum inner iterations
acc       k:   488     num_inner:    81     f: +1.921274e-02   |grad|: 9.790045e-01   maximum inner iterations
acc       k:   489     num_inner:    38     f: +1.920473e-02   |grad|: 8.267087e-02   reached target residual-kappa (linear)
acc       k:   490     num_inner:    81     f: +1.920183e-02   |grad|: 1.076205e-01   maximum inner iterations
acc       k:   491     num_inner:    81     f: +1.919892e-02   |grad|: 9.863974e-02   maximum inner iterations
acc       k:   492     num_inner:    81     f: +1.919605e-02   |grad|: 4.432258e-02   maximum inner iterations
acc       k:   493     num_inner:    81     f: +1.919298e-02   |grad|: 3.146952e-01   maximum inner iterations
acc       k:   494     num_inner:    72     f: +1.918998e-02   |grad|: 2.960419e-02   reached targ

7.542540073394775

In [254]:
np.linalg.matrix_rank(Q2)

14

In [239]:
W.shape

(1,)

In [189]:
U,S,V = np.linalg.svd(Y_star,full_matrices=False)
S

array([4.56106045, 3.2488941 , 3.10506275])

In [190]:
YY = np.diag(S)[:3,:]@V
YY = YY/ np.linalg.norm(YY,axis=0)
position

array([630.,   0., 585.])

In [193]:
direction_to_position(Y_star,C,D)

array([[ 2.60434843e-05,  4.97933347e+01,  4.54147023e+00],
       [ 2.60712579e-05, -4.54143646e+00,  4.97933380e+01],
       [ 5.00000069e+01, -7.67038961e-15,  6.71854709e-16],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 5.07776010e-05, -2.73173641e+01, -4.87725691e+01],
       [-6.69256602e-06, -2.26271622e+01,  5.11176292e+01],
       [ 7.20359175e+01,  5.27421211e+02, -7.48027822e+01],
       [ 6.94769051e+01,  5.32435411e+02,  2.50388948e+01],
       [ 2.79688106e+02,  5.30627118e+02, -1.95621076e+01],
       [ 3.29300186e+02,  5.36774477e+02, -1.85989112e+01],
       [ 5.80454054e+02,  6.08147130e+02, -4.37804456e+01],
       [ 5.88716052e+02,  5.31485842e+02,  1.98965738e+01],
       [ 5.84997990e+02,  5.95399977e+02,  1.88046547e+01],
       [ 5.85045460e+02,  6.27350969e+02,  5.72645639e+01],
       [ 5.84957192e+02,  6.28078619e+02,  5.79446958e+01]])

In [192]:
def direction_to_position(Y,C,D):
    X = np.linalg.pinv(C@C.transpose())@C@D@Y_star.transpose()
    X = X - X[3,:]
    
    if X[0,0]<0:
        X = -X
    
    v = X[2,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    
    return X@R.transpose()

In [450]:

v = np.array([ 4.99999990e+01, -1.75732342e-02, -7.92915769e-03])

R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))

In [110]:
v = R@Y_star[:,4] 

R1 = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/10)

In [158]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [54]:
X = np.linalg.pinv(C@C.transpose())@C@D@Y_star.transpose()
X

array([[-192.93037171,   91.86004079, -314.48272432,   23.69251499],
       [-220.73226786,   33.82940223, -340.35383563,   37.48321155],
       [-150.85343109,   39.1406749 , -334.77232541,   29.89712156],
       [-186.86418545,   66.24897017, -356.39589818,   30.8056396 ],
       [-188.31010643,   57.63109134, -362.48582007,   85.68326031],
       [-206.19111078,   34.49642711, -365.72338839,   -9.89183372],
       [-184.47966125,  223.50379141,  169.12011704,   26.79175869],
       [-200.43854953,  198.80969644,  166.3533098 ,  -68.74813261],
       [ 165.80946339,  -79.92695645,  227.54803638,   -7.31786382],
       [ 202.62642457, -113.12617493,  233.65111391,   -5.0662883 ],
       [ 181.487853  , -140.25868703,  226.71810499,  -40.6883725 ],
       [ 223.63433024,  -86.14002063,  240.50725253,   30.75839607],
       [ 229.66117739, -110.84740953,  253.45354491,  -26.56152468],
       [ 263.62788206, -107.94362046,  278.12756931,  -53.56194194],
       [ 263.95255345, -107.277225

In [55]:
X = - X + X[3,:]
X

array([[ 6.06618626e+00, -2.56110706e+01, -4.19131739e+01,
         7.11312461e+00],
       [ 3.38680824e+01,  3.24195679e+01, -1.60420626e+01,
        -6.67757195e+00],
       [-3.60107544e+01,  2.71082953e+01, -2.16235728e+01,
         9.08518041e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.44592098e+00,  8.61787883e+00,  6.08992189e+00,
        -5.48776207e+01],
       [ 1.93269253e+01,  3.17525431e+01,  9.32749021e+00,
         4.06974733e+01],
       [-2.38452420e+00, -1.57254821e+02, -5.25516015e+02,
         4.01388091e+00],
       [ 1.35743641e+01, -1.32560726e+02, -5.22749208e+02,
         9.95537722e+01],
       [-3.52673649e+02,  1.46175927e+02, -5.83943935e+02,
         3.81235034e+01],
       [-3.89490610e+02,  1.79375145e+02, -5.90047012e+02,
         3.58719279e+01],
       [-3.68352038e+02,  2.06507657e+02, -5.83114003e+02,
         7.14940121e+01],
       [-4.10498516e+02,  1.52388991e+02, -5.96903151e+02,
      

In [233]:
X[-1,:]-position

array([-1.07039074e+01, -4.44089210e-16, -6.63956294e-01])

In [67]:
R@X[-1,:]

array([620.0773281 ,  72.70829173, 987.47895214])

In [52]:
G.edges

EdgeView([('x', 'y'), ('x', 'a1_origin'), ('x', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a1_plus'), ('a1_origin', 'a1_plus'), ('a1_origin', 'a2_plus'), ('a1_origin', ' a2_minus'), ('a1_plus', 'a2_plus'), ('a1_plus', ' a2_minus'), ('a2_plus', ' a2_minus'), ('a2_plus', 'a3_plus'), ('a2_plus', 'a3_minus'), (' a2_minus', 'a3_plus'), (' a2_minus', 'a3_minus'), ('a3_plus', 'a3_minus'), ('a3_plus', 'a4_origin'), ('a3_plus', 'a4_minus'), ('a3_minus', 'a4_origin'), ('a3_minus', 'a4_minus'), ('a4_origin', 'a4_minus'), ('a4_origin', 'a5_plus'), ('a4_origin', 'a5_minus'), ('a4_minus', 'a5_plus'), ('a4_minus', 'a5_minus'), ('a5_plus', 'a5_minus'), ('a5_plus', 'a6_origin'), ('a5_plus', 'a6_minus'), ('a5_minus', 'a6_origin'), ('a5_minus', 'a6_minus'), ('a6_origin', 'a6_minus')])